In [ ]:
### imports

# external modules
import sys
import os
import matplotlib.pyplot as plt
import importlib

# local modules
sys.path.append(os.path.abspath('../../ML4DQMDC-PixelAE/omsapi'))
import get_oms_data
importlib.reload(get_oms_data)
from get_oms_data import get_oms_api, get_oms_data
from get_oms_data import get_oms_response_attributes, get_oms_response_attribute
sys.path.append(os.path.abspath('../../ML4DQMDC-PixelAE'))
import utils.plot_utils
importlib.reload(utils.plot_utils)
from utils.plot_utils import plot_hists

In [ ]:
# get the omsapi instance

omsapi = get_oms_api()

In [ ]:
# define run ranges

# references:
# - https://twiki.cern.ch/twiki/bin/view/CMS/PdmVRun3Analysis#2023_Era_definition
# - DAS (query runs for each ZeroBias DQMIO dataset)
eradict = {
  'Run2023C-v1': (367080, 367515), # (DAS: start from 367094)
  'Run2023C-v2': (367516, 367620), # (DAS: stop at 367758, but this is incorrect in DAS client, actual data does not overlap with v3)
  'Run2023C-v3': (367621, 367763), # (DAS: start at 367622, stop at 367758)
  'Run2023C-v4': (367765, 369802), # (DAS: start at 367770)
  'Run2023D-v1': (369803, 370602), # (DAS: start from 369827, stop at 370580)
  'Run2023D-v2': (370603, 372415), # (DAS: start from 370616, stop at 371227)
  'Run2023E': (372417, 373075), # (DAS: start from 372474, stop at 373072)
  'Run2023F': (373076, 373784) # (DAS: start from 373078, stop is from DAS as Twiki is not yet filled)
}

In [ ]:
# find out which attributes are available per lumisection

runnb = 367515 # dummy run, should not matter
ls_info = get_oms_data( omsapi, 'lumisections', runnb )
print( get_oms_response_attributes(ls_info) )

In [ ]:
# define attributes to retrieve

attributes = [
    'delivered_lumi_per_lumisection',
    'recorded_lumi_per_lumisection',
    'pileup',
    'physics_flag',
    'fill_number',
    'run_number',
    'lumisection_number',
]

In [ ]:
# get the data from OMS

def get_data_iterative(omsapi, runrange, attributes):
    # note: the maximum allowed limit in OMS calls seems to be 100k lumisections,
    #       but some eras have more, so need to split run range in parts.
    limit_entries = 100000 # hard limit in OMS API
    ls_info_probe = get_oms_data( omsapi, 'lumisections', runrange, attributes=['lumisection_number'], limit_entries=limit_entries )
    if len(get_oms_response_attribute(ls_info_probe,'lumisection_number')) < limit_entries:
        ls_info_raw = get_oms_data( omsapi, 'lumisections', runrange, attributes=attributes, limit_entries=limit_entries )
        ls_info = {attribute: get_oms_response_attribute(ls_info_raw, attribute) for attribute in attributes}
        return ls_info
    else:
        midrange = int((runrange[0]+runrange[1])/2)
        firstrange = (runrange[0], midrange)
        ls_info_1 = get_data_iterative(omsapi, firstrange, attributes)
        secondrange = (midrange+1, runrange[1])
        ls_info_2 = get_data_iterative(omsapi, secondrange, attributes)
        ls_info = {attribute: ls_info_1[attribute]+ls_info_2[attribute] for attribute in attributes}
        return ls_info

info = {}
for era, runrange in eradict.items():
    print('Retrieving data for era {}'.format(era))
    ls_info = get_data_iterative(omsapi, runrange, attributes)
    print('Found {} lumisections'.format(len(ls_info[list(ls_info.keys())[0]])))
    info[era] = ls_info

In [ ]:
# make plots
# note: the unit of luminosity per lumisection is inverse pb, as retrieved from looking at OMS tables.

for era in eradict.keys():
    fig, ax = plt.subplots(figsize=(18,6))
    thisinfo = info[era]
    plot_hists([thisinfo['delivered_lumi_per_lumisection'], thisinfo['recorded_lumi_per_lumisection']],
               fig=fig, ax=ax,
               colorlist=['b','g'],
               labellist=['Delivered lumi', 'Recorded lumi'],
               xaxtitle='Lumisection', xaxtitlesize=15,
               yaxtitle='Luminosity (/pb)', yaxtitlesize=15,
               title='Luminosity for era {}'.format(era), titlesize=15,
               ymaxfactor=1.2, legendsize=15, opaque_legend=False, ticksize=None,
               bkgcolor=thisinfo['physics_flag'], bkgcmap='spring')
    fig, ax = plt.subplots(figsize=(18,6))
    plot_hists([thisinfo['pileup']],
               fig=fig, ax=ax,
               colorlist=['b','g'],
               labellist=['Pileup'],
               xaxtitle='Lumisection', xaxtitlesize=15,
               yaxtitle='Pileup', yaxtitlesize=15,
               title='Pileup for era {}'.format(era), titlesize=15,
               ymaxfactor=1.2, legendsize=15, opaque_legend=False, ticksize=None,
               bkgcolor=thisinfo['physics_flag'], bkgcmap='spring')

In [ ]:
# save to json files
import json

for era in eradict.keys():
    thisinfo = info[era]
    outputfile = 'omsdata/omsdata_{}.json'.format(era)
    with open(outputfile, 'w') as f:
        json.dump(thisinfo, f)